## Segmenting and Clustering Neighborhoods in Toronto

The goal of this projects is to cluster neighborhoods of Toronto, which have similar characteristics, based on the data retrieved from the Foursquare API.

The steps will include:

1) Retrieving neighborhoods data from the Toronto Wikipedia page through BeautifulSoup

2) Explore Neighbourhoods in Toronto

3) Analyze Each Neighbourhood

4) Clusering

5) Examine Cluster


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="##item1">Retrieving neighborhoods data from the Toronto Wikipedia page through BeautifulSoup</a>

2.  <a href="##item2">Explore Neighborhoods in Toronto</a>

3.  <a href="##item3">Analyze Each Neighborhood</a>

4.  <a href="##item4">Clustering</a>

5.  <a href="##item5">Examine Cluster</a>  
    </font>
    </div>


In [1]:
# pip install geopy
# pip install folium

In [2]:
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.geocoders import ArcGIS

import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Retrieving neighborhoods data from the Toronto Wikipedia page through BeautifulSoup

In [3]:
url_wiki = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"

page = requests.get(url_wiki)

soup = BeautifulSoup(page.content, 'html.parser')


In [4]:
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [5]:
toronto_table = soup.find_all("table", class_="wikitable sortable")

In [6]:
toronto_table[0]


<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a class="mw-redirect" href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></t

In [7]:
header = [th.text.rstrip() for th in toronto_table[0].find_all("th")]
header

['Postcode', 'Borough', 'Neighbourhood']

In [8]:
toronto_table[0].find_all("tr")[0].find_all("td")

[]

In [9]:
c1 = []
c2 = []
c3 = []


for row in toronto_table[0].find_all("tr"):


    cells = row.find_all("td")
    
    if (cells):
        
        
        c1.append(cells[0].find(text=True))
        c2.append(cells[1].find(text=True))
        c3.append(cells[2].find(text=True).split("\n")[0])

In [10]:
toronto_df = pd.DataFrame(columns=header)

In [11]:
toronto_df["Postcode"] = c1
toronto_df["Borough"] = c2
toronto_df["Neighbourhood"] = c3

In [12]:
toronto_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [13]:
drop_index = []
for index, row in toronto_df.iterrows():
    
    if row[1] == "Not assigned":
        toronto_df.drop(index, inplace=True)
    

In [14]:
toronto_df.reset_index(inplace=True, drop=True)

In [15]:
toronto_df = toronto_df.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda x: ','.join(x)).reset_index()

In [16]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
geolocator= ArcGIS()

for index, row in toronto_df.iterrows():
    postcode = toronto_df.loc[index, "Postcode"]
    g=geocoder.arcgis('{}, Toronto, Ontario'.format(postcode))
    toronto_df.at[index, "Latitude"] = g.latlng[0]
    toronto_df.at[index, "Longitude"] = g.latlng[1]

In [18]:
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.81139,-79.19662
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [19]:
address = "Toronto, Ontario"
geo = Nominatim(user_agent="toronto_explorer")
location = geo.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinate of Toronto are {}, {}".format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347


In [20]:
from folium.plugins import MarkerCluster


In [21]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

neighbors = MarkerCluster().add_to(toronto_map)


for lat, long, bor, pc in zip(toronto_df.Latitude, toronto_df.Longitude, toronto_df.Borough, toronto_df.Postcode):
    
    label = "{}, {}".format(bor, pc)
    
    folium.Marker(
        location = [lat, long],
        icon = None,
        popup = label,
    ).add_to(neighbors)

toronto_map

In [22]:
index = toronto_df["Borough"].str.contains("Toronto")
subset = toronto_df.loc[index, :]
print(subset.shape)
split = subset["Neighbourhood"].str.split(",")
lens = split.str.len()

subset = pd.DataFrame({"Postcode": np.repeat(subset["Postcode"].values, lens),
                       "Borough": np.repeat(subset["Borough"].values, lens),
                       "Neighbourhood" : np.hstack(split)
                      })

(38, 5)


In [23]:
subset.head()

,Postcode,Borough,Neighbourhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,The Danforth West
2,M4K,East Toronto,Riverdale
3,M4L,East Toronto,The Beaches West
4,M4L,East Toronto,India Bazaar


In [24]:
geolocator= ArcGIS()

for index, row in subset.iterrows():
    neighbourhood = subset.loc[index, "Neighbourhood"]
    g=geocoder.arcgis('{}, Toronto, Ontario'.format(neighbourhood))
    subset.at[index, "Latitude"] = g.latlng[0]
    subset.at[index, "Longitude"] = g.latlng[1]

In [25]:
subset.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.67413,-79.296440
1,M4K,East Toronto,The Danforth West,43.69067,-79.290792
2,M4K,East Toronto,Riverdale,43.67323,-79.341210
3,M4L,East Toronto,The Beaches West,43.67413,-79.296440
4,M4L,East Toronto,India Bazaar,43.73840,-79.565240


In [26]:
subset_map = folium.Map(location=[latitude, longitude], zoom_start=12)

neighbors = MarkerCluster().add_to(subset_map)


for lat, long, bor, neigh in zip(subset.Latitude, subset.Longitude, subset.Borough, subset.Neighbourhood):
    
    label = "{}, {}".format(bor, neigh)
    
    folium.Marker(
        location = [lat, long],
        icon = None,
        popup = label,
    ).add_to(neighbors)

subset_map

In [27]:
CLIENT_ID = 'R3JBLNFG4YWVLI32QAUWLT3H1BXLZM4F2JANDE5CRWNSM0DX' # your Foursquare ID
CLIENT_SECRET = 'CBQVHLXRPSMBVIWC2TJEA3ZGCOHIPUTRVPNNAXKZICR2QA31' # your Foursquare Secret
ACCESS_TOKEN = "5NDXXPAHKA53GHI32GR2BMJF1KFO5XKSECS5ERAPT52ERGAJ"
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R3JBLNFG4YWVLI32QAUWLT3H1BXLZM4F2JANDE5CRWNSM0DX
CLIENT_SECRET:CBQVHLXRPSMBVIWC2TJEA3ZGCOHIPUTRVPNNAXKZICR2QA31
ACCESS_TOKEN:CBQVHLXRPSMBVIWC2TJEA3ZGCOHIPUTRVPNNAXKZICR2QA31


In [28]:
neighborhood_latitude = round(subset.loc[0, 'Latitude'], 2) # neighborhood latitude value
neighborhood_longitude = round(subset.loc[0, 'Longitude'], 2) # neighborhood longitude value

neighborhood_name = subset.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))
                   

Latitude and longitude values of The Beaches are 43.67, -79.3.


In [29]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=R3JBLNFG4YWVLI32QAUWLT3H1BXLZM4F2JANDE5CRWNSM0DX&client_secret=CBQVHLXRPSMBVIWC2TJEA3ZGCOHIPUTRVPNNAXKZICR2QA31&v=20180605&ll=43.67,-79.3&radius=500&limit=100'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606ac98825fa6e064b8d9c43'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 62,
  'suggestedBounds': {'ne': {'lat': 43.674500004500004,
    'lng': -79.29379037233618},
   'sw': {'lat': 43.6654999955, 'lng': -79.30620962766382}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae1ed7f964a520ba863be3',
       'name': 'Kew Gardens',
       'location': {'address': '2075 Queen Street East',
        'crossStreet': 'btwn Waverly Rd. & Lee Ave.',
        'lat': 43.66903821389327,
        'lng': -79.29853750338029,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.66903821389327,
          'lng': -79.2985375033802

In [31]:
def extract_category(row):
    try:
        category = row["categories"]
    
    except:
        category = row["venue.categories"]
        
    if len(category) == 0:
        return None
    
    else:
        return category[0]["name"]

In [32]:
venues = results["response"]["groups"][0]["items"]
venues = json_normalize(venues)

filtered_columns = ["venue.name", "venue.categories", "venue.location.lat", "venue.location.lng"]

filtered_venues = venues.loc[:, filtered_columns]

filtered_venues["venue.categories"] = filtered_venues.apply(extract_category, axis = 1)

filtered_venues.columns = [col.split(".")[-1] for col in filtered_venues.columns]

<ipython-input-32-1785037c35bc>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  venues = json_normalize(venues)


In [33]:
results["response"]["groups"][0]["items"][1]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5279396111d2c5664928c346',
  'name': 'Hogtown Smoke',
  'location': {'address': '1959 Queen St E',
   'crossStreet': 'at Kenilworth Ave',
   'lat': 43.669573021423744,
   'lng': -79.30215071863523,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.669573021423744,
     'lng': -79.30215071863523}],
   'distance': 179,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['1959 Queen St E (at Kenilworth Ave)',
    'Toronto ON',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1df931735',
    'name': 'BBQ Joint',
    'pluralName': 'BBQ Joints',
    'shortName': 'BBQ',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5279396111d2c566

In [34]:
filtered_venues.head()

,name,categories,lat,lng
0,Kew Gardens,Park,43.669038,-79.298538
1,Hogtown Smoke,BBQ Joint,43.669573,-79.302151
2,Buds Coffee Bar,Coffee Shop,43.669375,-79.303218
3,Kew Gardens Play Park,Park,43.669267,-79.299684
4,The Ten Spot,Nail Salon,43.670340,-79.299363


In [35]:
subset.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.674130,-79.296440
1,M4K,East Toronto,The Danforth West,43.690670,-79.290792
2,M4K,East Toronto,Riverdale,43.673230,-79.341210
3,M4L,East Toronto,The Beaches West,43.674130,-79.296440
4,M4L,East Toronto,India Bazaar,43.738400,-79.565240
5,M4M,East Toronto,Studio District,43.648690,-79.385440
6,M4N,Central Toronto,Lawrence Park,43.725540,-79.402320
7,M4P,Central Toronto,Davisville North,43.700634,-79.385359
8,M4R,Central Toronto,North Toronto West,43.724000,-79.401980
9,M4S,Central Toronto,Davisville,43.701750,-79.383520


## 2. Explore Neighbourhoods in Toronto

In [53]:
def getNearbyVenues(database, radius = 500):
    
    # type your answer here
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    CLIENT_ID = 'B53BNE4XAPOGU3CMODGRBHB1BRFAV2LN5PURBFP4AR3PUEBG' # your Foursquare ID
    CLIENT_SECRET = '1XWWPHJFNN3Q141V0AZYW3CUQBEP3CL4KS2P2IVCSRJMNUOI' # your Foursquare Secret
    VERSION = '20210304' # Foursquare API version
    
    
    venues_list = []
    for neigh, lat, long in zip(database["Neighbourhood"], database["Latitude"], database["Longitude"]):
#         print(neigh)
        # create URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET, 
            VERSION,
            lat, 
            long, 
            radius, 
            LIMIT)
        
        result = requests.get(url).json()
        
        venues = result["response"]["groups"][0]["items"]
        
        for v in venues:
            
            venues_list.append({"Neighbourhood Name": neigh, 
                              "Neighbourhood Latitude": lat, 
                              "Neighbourhood Longitude": long, 
                              "Venue Name": v["venue"]["name"], 
                              "Venue Latitude": v["venue"]["location"]["lat"], 
                              "Venue Longitude": v["venue"]["location"]["lng"],
                              "Venue Category": v["venue"]["categories"][0]["name"]
                             })
            
        venues_df = pd.DataFrame(venues_list)
            
    return venues_df 
        
        
    
    

In [54]:
venues_df = getNearbyVenues(subset)

In [55]:
venues_df.head(10)

,Neighbourhood Name,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.67413,-79.29644,Mastermind Toys,43.671453,-79.293971,Toy / Game Store
1,The Beaches,43.67413,-79.29644,Glen Manor Ravine,43.676821,-79.293942,Trail
2,The Beaches,43.67413,-79.29644,Sanna's Farmacia,43.670929,-79.295969,Juice Bar
3,The Beaches,43.67413,-79.29644,The Ten Spot,43.670340,-79.299363,Nail Salon
4,The Beaches,43.67413,-79.29644,Yumei Sushi,43.671108,-79.295064,Japanese Restaurant
5,The Beaches,43.67413,-79.29644,Castro's Lounge,43.671104,-79.295107,Bar
6,The Beaches,43.67413,-79.29644,Green Eggplant,43.670517,-79.298660,Mediterranean Restaurant
7,The Beaches,43.67413,-79.29644,Pippins Tea Company,43.670992,-79.295905,Tea Room
8,The Beaches,43.67413,-79.29644,Beacher Cafe,43.671938,-79.291238,Breakfast Spot
9,The Beaches,43.67413,-79.29644,The Nutty Chocolatier,43.670567,-79.297598,Chocolate Shop


## 3. Analyze Each Neighbourhood

In [56]:
toronto_one_hot = pd.get_dummies(venues_df["Venue Category"])
toronto_one_hot.insert(0, "Neighbourhood", venues_df["Neighbourhood Name"])

In [57]:
toronto_one_hot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tunnel,Udon Restaurant,Univ

In [58]:
toronto_grouped = toronto_one_hot.groupby("Neighbourhood").mean().reset_index()

In [79]:
toronto_grouped.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tunnel,Udon Restaurant,Univ

In [74]:
for hood in toronto_grouped["Neighbourhood"]:
    print("----------",hood,"-----------")
    temp = toronto_grouped[toronto_grouped["Neighbourhood"] ==hood].T.reset_index()
    temp.columns = ["Category", "freq"]
    temp = temp.iloc[1:]
    temp["freq"] = temp["freq"].astype(float)
    temp = temp.round({"freq": 2})
    print(temp.sort_values("freq", ascending=False).reset_index(drop=True).head(5))
    
    
    
    
    
    
    
    

---------- Adelaide -----------
                 Category  freq
0             Coffee Shop  0.06
1  Furniture / Home Store  0.06
2        Asian Restaurant  0.04
3                     Spa  0.04
4             Yoga Studio  0.03
---------- Bathurst Quay -----------
        Category  freq
0    Coffee Shop  0.19
1  Grocery Store  0.07
2           Park  0.07
3           Café  0.07
4         Garden  0.04
---------- Berczy Park -----------
              Category  freq
0          Coffee Shop  0.08
1                 Café  0.05
2                Hotel  0.04
3  Japanese Restaurant  0.04
4   Seafood Restaurant  0.04
---------- Brockton -----------
                Category  freq
0            Coffee Shop  0.12
1                   Café  0.10
2                    Bar  0.10
3  Vietnamese Restaurant  0.05
4          Grocery Store  0.05
---------- Business Reply Mail Processing Centre 969 Eastern -----------
               Category  freq
0           Yoga Studio  0.05
1               Butcher  0.05
2  Gym / Fi

In [92]:
columns = ["Neighbourhood"]

n_top_venues = 10

suffix = ["st", "nd", "rd"]


for n in range(1, n_top_venues+1):
    
    try:
        columns.append("{}{} Most Common Venue".format(n, suffix[n-1]))
        
    except:
        columns.append("{}th Most Common Venue".format(n))


neighbourhoods_sorted = pd.DataFrame(columns=columns)

neighbourhoods_sorted["Neighbourhood"] = toronto_grouped["Neighbourhood"]

for ind in range(0, len(toronto_grouped["Neighbourhood"])):
    neighbourhoods_sorted.iloc[ind, 1:] = toronto_grouped.iloc[ind, 1:].sort_values(ascending=False).index.values[0:n_top_venues]





In [93]:
neighbourhoods_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Furniture / Home Store,Asian Restaurant,Spa,Yoga Studio,Restaurant,Sandwich Place,Italian Restaurant,Gym / Fitness Center,Boutique
1,Bathurst Quay,Coffee Shop,Grocery Store,Park,Café,Garden,Gym,Harbor / Marina,Sculpture Garden,Sushi Restaurant,Boutique
2,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Seafood Restaurant,Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Gastropub
3,Brockton,Coffee Shop,Café,Bar,Vietnamese Restaurant,Grocery Store,Middle Eastern Restaurant,Boutique,Bakery,Pizza Place,Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Butcher,Gym / Fitness Center,Grocery Store,Beer Store,Garden Center,Garden,Spa,Burrito Place,Clothing Store


## 4. Clustering

In [102]:
k = 5

cluster_df = toronto_grouped.drop("Neighbourhood", 1)
clz = KMeans(n_clusters = k, random_state = 0).fit(cluster_df)

clz.labels_[0:10]






array([0, 0, 0, 0, 2, 0, 0, 0, 0, 3])

In [111]:
neighbourhoods_sorted.insert(0, "Cluster", clz.labels_)

toronto_merged = subset

toronto_merged = toronto_merged.join(neighbourhoods_sorted.set_index("Neighbourhood"), on="Neighbourhood")

In [121]:
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.67413,-79.296440,0,Japanese Restaurant,Thai Restaurant,Bar,Breakfast Spot,Toy / Game Store,Burger Joint,Nail Salon,Café,Mobile Phone Shop,Chocolate Shop
1,M4K,East Toronto,The Danforth West,43.69067,-79.290792,0,Grocery Store,Bank,Ice Cream Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Hardware Store,Metro Station,Gym,Beer Store
2,M4K,East Toronto,Riverdale,43.67323,-79.341210,1,Fast Food Restaurant,Caribbean Restaurant,Department Store,Café,Sandwich Place,Dog Run,Accessories Store,Other Great Outdoors,Organic Grocery,Optical Shop
3,M4L,East Toronto,The Beaches West,43.67413,-79.296440,0,Japanese Restaurant,Thai Restaurant,Bar,Breakfast Spot,Toy / Game Store,Burger Joint,Nail Salon,Café,Mobile Phone Shop,Chocolate Shop
4,M4L,East Toronto,India Bazaar,43.73840,-79.565240,2,Indian Restaurant,Caribbean Restaurant,Ice Cream Shop,American Restaurant,Coffee Shop,Pizza Place,Bank,Neighborhood,New American Restaurant,Nightclub


In [136]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors  

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)


# colors = plt.cm.rainbow(np.linspace(0, 1, k))

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

map_clusters

## 5. Examine Cluster

## Cluster 1

In [139]:
toronto_merged.loc[toronto_merged['Cluster'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Japanese Restaurant,Thai Restaurant,Bar,Breakfast Spot,Toy / Game Store,Burger Joint,Nail Salon,Café,Mobile Phone Shop,Chocolate Shop
1,East Toronto,0,Grocery Store,Bank,Ice Cream Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Hardware Store,Metro Station,Gym,Beer Store
3,East Toronto,0,Japanese Restaurant,Thai Restaurant,Bar,Breakfast Spot,Toy / Game Store,Burger Joint,Nail Salon,Café,Mobile Phone Shop,Chocolate Shop
5,East Toronto,0,Coffee Shop,Hotel,Café,Asian Restaurant,Seafood Restaurant,Thai Restaurant,Taco Place,Sushi Restaurant,Steakhouse,Concert Hall
6,Central Toronto,0,Coffee Shop,Bank,Italian Restaurant,BBQ Joint,Frozen Yogurt Shop,Spa,Bubble Tea Shop,Neighborhood,Gastropub,Mobile Phone Shop
7,Central Toronto,0,Coffee Shop,Sandwich Place,Indian Restaurant,Gas Station,Tennis Court,Thai Restaurant,Cemetery,Mexican Restaurant,Fast Food Restaurant,Park
8,Central Toronto,0,Ice Cream Shop,Supermarket,Bank,Thai Restaurant,Coffee Shop,Bus Line,Gastropub,Garden,Pool,Nail Salon
9,Central Toronto,0,Sandwich Place,Indian Restaurant,Gym,Dessert Shop,Diner,Restaurant,Coffee Shop,Park,Fast Food Restaurant,Café
12,Central Toronto,0,Sandwich Place,Coffee Shop,History Museum,Café,Museum,Flower Shop,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Steakhouse
14,Central Toronto,0,French Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Flower Shop,Café,Shoe Repair,Park,Coffee Shop,Pet Store,Pub


## Cluster 2

In [141]:
toronto_merged.loc[toronto_merged['Cluster'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,1,Fast Food Restaurant,Caribbean Restaurant,Department Store,Café,Sandwich Place,Dog Run,Accessories Store,Other Great Outdoors,Organic Grocery,Optical Shop


## Cluster 3

In [142]:
toronto_merged.loc[toronto_merged['Cluster'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,2,Indian Restaurant,Caribbean Restaurant,Ice Cream Shop,American Restaurant,Coffee Shop,Pizza Place,Bank,Neighborhood,New American Restaurant,Nightclub
10,Central Toronto,2,Middle Eastern Restaurant,Korean Restaurant,Print Shop,Board Shop,Park,Coffee Shop,Discount Store,Rental Car Location,Indian Restaurant,Bar
39,Central Toronto,2,Bank,Spa,Trail,Pizza Place,Accessories Store,New American Restaurant,Optical Shop,Opera House,Office,Noodle House
44,Downtown Toronto,2,Korean Restaurant,Grocery Store,Coffee Shop,Dessert Shop,Japanese Restaurant,Karaoke Bar,Pizza Place,Bubble Tea Shop,Fried Chicken Joint,Café
67,West Toronto,2,Light Rail Station,Amphitheater,Other Great Outdoors,Dog Run,Playground,Curling Ice,Music Venue,Nail Salon,Neighborhood,New American Restaurant
72,West Toronto,2,Gym / Fitness Center,Light Rail Station,Park,Bus Stop,Massage Studio,Lake,Trail,Gas Station,Deli / Bodega,Wine Shop
73,East Toronto,2,Yoga Studio,Butcher,Gym / Fitness Center,Grocery Store,Beer Store,Garden Center,Garden,Spa,Burrito Place,Clothing Store


## Cluster 4

In [143]:
toronto_merged.loc[toronto_merged['Cluster'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Central Toronto,3,Park,Indian Restaurant,Pet Store,Breakfast Spot,Sushi Restaurant,Café,Beer Store,Thai Restaurant,Bank,Grocery Store
13,Central Toronto,3,Field,Convenience Store,Trail,Dog Run,Farmers Market,Office,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House
15,Central Toronto,3,Park,Trail,Coffee Shop,Bank,Sandwich Place,Pie Shop,Neighborhood,New American Restaurant,Nightclub,Noodle House
16,Central Toronto,3,Park,Indian Restaurant,Pet Store,Breakfast Spot,Sushi Restaurant,Café,Beer Store,Thai Restaurant,Bank,Grocery Store
29,Downtown Toronto,3,Convenience Store,Jewelry Store,Park,Moving Target,Nightclub,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House,Office
38,Central Toronto,3,Grocery Store,Men's Store,Convenience Store,Park,Gym Pool,Caribbean Restaurant,Outdoor Sculpture,Other Great Outdoors,Organic Grocery,Optical Shop
59,Downtown Toronto,3,Park,Grocery Store,Café,Italian Restaurant,American Restaurant,Nightclub,Coffee Shop,Gym,Beer Store,Candy Store
61,West Toronto,3,Pharmacy,Pizza Place,Bank,Park,Coffee Shop,Bakery,Grocery Store,Food Truck,Neighborhood,New American Restaurant
71,West Toronto,3,Park,Grocery Store,Gas Station,Intersection,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House,Office,Noodle House


## Cluster 5

In [146]:
toronto_merged.loc[toronto_merged['Cluster'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Central Toronto,4,Skating Rink,Gym / Fitness Center,Shopping Mall,Accessories Store,Noodle House,Organic Grocery,Optical Shop,Opera House,Office,Nightclub
